In [1]:
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised
from helper import series_to_supervised, stage_series_to_supervised

2024-06-08 20:26:52.185827: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
dataset = pd.read_csv('../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
dataset

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
print(dataset.columns)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')


### Add 40% noise to rainfall and tide (WS_S4) in the test set

In [26]:
random_list = []

positive_error_bound = 40
negative_error_bound = -40

for i in range(0, len(dataset)):
    random_list.append(random.randint(-negative_error_bound, positive_error_bound) / 100)
    
# print(random_list)

dataset['noise'] = random_list
dataset['Noise_MEAN_RAIN'] = dataset['MEAN_RAIN'] + dataset['MEAN_RAIN']*dataset['noise']
dataset['Noise_WS_S4'] = dataset['WS_S4'] + dataset['WS_S4']*dataset['noise']
dataset

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,...,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,noise,Noise_MEAN_RAIN,Noise_WS_S4,New_WS_S4,New_MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,...,0.00,2.78,0.505,0.0,0.0,0.2,0.0,0.408,0.340,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,...,0.00,2.78,-0.110,0.0,0.0,0.2,0.0,-0.336,-0.280,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.650,0.0,0.0,0.2,0.0,-0.876,-0.730,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,...,0.00,2.79,-0.850,0.0,0.0,0.2,0.0,-1.020,-0.850,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.550,0.0,0.0,0.2,0.0,-0.588,-0.490,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,...,2.61,2.49,1.290,0.0,0.0,0.2,0.0,1.428,1.666,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,...,2.62,2.48,1.660,0.0,0.0,0.2,0.0,1.932,2.254,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,...,2.62,2.54,2.040,0.0,0.0,0.2,0.0,2.280,2.660,0.0


In [27]:
split = int(len(dataset) * 0.8)
split

77145

In [28]:
# add 40% random noise to tide: WS_S4
WS_S4 = dataset['WS_S4']
first_80_WS_S4 = WS_S4[:split]

Noise_WS_S4 = dataset['Noise_WS_S4']
last_20_Noise_WS_S4 = Noise_WS_S4[split:]

dataset['New_WS_S4'] = pd.concat([first_80_WS_S4, last_20_Noise_WS_S4], axis=0)
dataset

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,...,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,noise,Noise_MEAN_RAIN,Noise_WS_S4,New_WS_S4,New_MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,...,0.00,2.78,0.505,0.0,0.0,0.2,0.0,0.408,0.340,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,...,0.00,2.78,-0.110,0.0,0.0,0.2,0.0,-0.336,-0.280,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.650,0.0,0.0,0.2,0.0,-0.876,-0.730,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,...,0.00,2.79,-0.850,0.0,0.0,0.2,0.0,-1.020,-0.850,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.550,0.0,0.0,0.2,0.0,-0.588,-0.490,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,...,2.61,2.49,1.290,0.0,0.0,0.2,0.0,1.428,1.428,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,...,2.62,2.48,1.660,0.0,0.0,0.2,0.0,1.932,1.932,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,...,2.62,2.54,2.040,0.0,0.0,0.2,0.0,2.280,2.280,0.0


In [29]:
# add 40% random noise to rainfall: MEAN_RAIN
MEAN_RAIN = dataset['MEAN_RAIN']
first_80_MEAN_RAIN = MEAN_RAIN[:split]

Noise_MEAN_RAIN = dataset['Noise_MEAN_RAIN']
last_20_Noise_MEAN_RAIN = Noise_MEAN_RAIN[split:]

dataset['New_MEAN_RAIN'] = pd.concat([first_80_MEAN_RAIN, last_20_Noise_MEAN_RAIN], axis=0)
dataset

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,...,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,noise,Noise_MEAN_RAIN,Noise_WS_S4,New_WS_S4,New_MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,...,0.00,2.78,0.505,0.0,0.0,0.2,0.0,0.408,0.340,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,...,0.00,2.78,-0.110,0.0,0.0,0.2,0.0,-0.336,-0.280,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.650,0.0,0.0,0.2,0.0,-0.876,-0.730,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,...,0.00,2.79,-0.850,0.0,0.0,0.2,0.0,-1.020,-0.850,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,...,0.00,2.79,-0.550,0.0,0.0,0.2,0.0,-0.588,-0.490,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,...,2.61,2.49,1.290,0.0,0.0,0.2,0.0,1.428,1.428,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,...,2.62,2.48,1.660,0.0,0.0,0.2,0.0,1.932,1.932,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,...,2.62,2.54,2.040,0.0,0.0,0.2,0.0,2.280,2.280,0.0


In [30]:
print(dataset.columns)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'noise', 'Noise_MEAN_RAIN',
       'Noise_WS_S4', 'New_WS_S4', 'New_MEAN_RAIN'],
      dtype='object')


In [31]:
# replace original WS_S4, MEAN_RAIN by New_WS_S4, New_MEAN_RAIN

data = dataset[['WS_S1', 'New_WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'New_MEAN_RAIN']]
data



# data = dataset[['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
#        'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
#        'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
#        'TWS_S26', 'PUMP_S26', 'New_MEAN_RAIN']]
# data

,WS_S1,New_WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,New_MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.340,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.280,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.730,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.850,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.490,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.428,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.932,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,2.280,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [32]:
data.columns

Index(['WS_S1', 'New_WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'New_MEAN_RAIN'],
      dtype='object')

### Do the same thing as before

In [33]:
# specify the number of lag hours
n_hours = 24*7
K = 24

In [34]:
# Pre-processing
# Stage --> 8 stages
stages = dataset[['WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26']]
print("stages.shape:", stages.shape)


stages_supervised = series_to_supervised(stages, n_hours, K)
print("stages_supervised.shape:", stages_supervised.shape)

stages.shape: (96432, 4)
stages_supervised.shape: (96241, 768)


In [35]:
# non_stages = data[['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'FLOW_S25B', 'GATE_S25B', 'FLOW_S26', 'GATE_S26', 'PUMP_S26', 'PUMP_S25B', 'MEAN_RAIN']]
non_stages = data[['New_WS_S4', 'FLOW_S25A', 'FLOW_S25B', 'FLOW_S26', 'PUMP_S26', 'PUMP_S25B', 'New_MEAN_RAIN']]
print("non_stages.shape:", non_stages.shape)

non_stages_supervised = series_to_supervised(non_stages, n_hours-1, 1)
print("non_stages_supervised.shape:", non_stages_supervised.shape)

non_stages.shape: (96432, 7)
non_stages_supervised.shape: (96265, 1176)


In [36]:
non_stages_supervised_cut = non_stages_supervised.iloc[K:, :]
print("non_stages_supervised_cut.shape:", non_stages_supervised_cut.shape)

non_stages_supervised_cut.shape: (96241, 1176)


In [37]:
# specify the number of lag hours
# n_hours = 72, stages_head.shape[1] +
n_features = stages.shape[1] + non_stages.shape[1]   # 1 rainfall + FGate_S25A + FGate_S25B + FGate_S26 + 8WS + PUMP_S26
# K = 12

In [38]:
# stages_head_supervised.reset_index(drop=True, inplace=True)
non_stages_supervised_cut.reset_index(drop=True, inplace=True)
stages_supervised.reset_index(drop=True, inplace=True)

all_data = concat([
                   non_stages_supervised_cut.iloc[:, :],
#                    stages_head_supervised.iloc[:, :-3*K],
                   stages_supervised.iloc[:, :]],
                   axis=1)

In [39]:
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (96241, 1944)


In [40]:
all_data = all_data.values
n_train_hours = int(len(all_data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]
test = all_data[n_train_hours:, :]

n_train_hours: 76992


In [41]:
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -stages.shape[1]*K:]
test_X, test_y = test[:, :n_obs], test[:, -stages.shape[1]*K:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (76992, 1848) (76992, 96) (19249, 1848) (19249, 96)


In [42]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (76992, 168, 11) (76992, 96) (19249, 168, 11) (19249, 96)


### Import trained model and test the performance with noised tide and rainfall

In [22]:
from tensorflow.keras.models import load_model
models = ['mlp', 'rnn', 'lstm', 'cnn', 'rcnn_24h']

for i in range(len(models)):
    print("========= {} =========".format(models[i]))
    saved_model = load_model('saved_model/{}.h5'.format(models[i]))
    yhat = saved_model.predict(test_X)
    inv_yhat = scaler.inverse_transform(yhat)
    inv_y = scaler.inverse_transform(test_y)
    inv_yhat = pd.DataFrame(inv_yhat)
    inv_y = pd.DataFrame(inv_y)
    error = inv_y - inv_yhat
    print('1h MAE = {}'.format(float("{:.6f}".format(mae(inv_yhat.iloc[:, 0:4], inv_y.iloc[:, 0:4])))))
    print('1h RMSE = {}'.format(float("{:.6f}".format(sqrt(mse(inv_yhat.iloc[:, 0:4], inv_y.iloc[:, 0:4]))))))
    print('8h MAE = {}'.format(float("{:.6f}".format(mae(inv_yhat.iloc[:, 28:32], inv_y.iloc[:, 28:32])))))
    print('8h RMSE = {}'.format(float("{:.6f}".format(sqrt(mse(inv_yhat.iloc[:, 28:32], inv_y.iloc[:, 28:32]))))))
    print('16h MAE = {}'.format(float("{:.6f}".format(mae(inv_yhat.iloc[:, 60:64], inv_y.iloc[:, 60:64])))))
    print('16h RMSE = {}'.format(float("{:.6f}".format(sqrt(mse(inv_yhat.iloc[:, 60:64], inv_y.iloc[:, 60:64]))))))
    print('24h MAE = {}'.format(float("{:.6f}".format(mae(inv_yhat.iloc[:, 92:96], inv_y.iloc[:, 92:96])))))
    print('24h RMSE = {}'.format(float("{:.6f}".format(sqrt(mse(inv_yhat.iloc[:, 92:96], inv_y.iloc[:, 92:96]))))))
    print('All MAE = {}'.format(float("{:.6f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
    print('All RMSE = {}'.format(float("{:.6f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

========= mlp =========


2024-06-08 20:27:08.995787: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-06-08 20:27:08.997926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-06-08 20:27:09.027908: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:85:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2024-06-08 20:27:09.027942: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-06-08 20:27:09.059897: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-06-08 20:27:09.059943: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

1h MAE = 0.154491
1h RMSE = 0.19291
8h MAE = 0.149032
8h RMSE = 0.19325
16h MAE = 0.146533
16h RMSE = 0.181807
24h MAE = 0.173612
24h RMSE = 0.21329
All MAE = 0.156804
All RMSE = 0.197375
========= rnn =========
1h MAE = 0.078709
1h RMSE = 0.10183
8h MAE = 0.109518
8h RMSE = 0.143415
16h MAE = 0.128279
16h RMSE = 0.169782
24h MAE = 0.128727
24h RMSE = 0.170079
All MAE = 0.114898
All RMSE = 0.152182
========= lstm =========
1h MAE = 0.166497
1h RMSE = 0.208907
8h MAE = 0.279815
8h RMSE = 0.356633
16h MAE = 0.243323
16h RMSE = 0.302895
24h MAE = 0.184311
24h RMSE = 0.235125
All MAE = 0.239585
All RMSE = 0.303196
========= cnn =========


2024-06-08 20:27:33.702163: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


1h MAE = 0.084929
1h RMSE = 0.109272
8h MAE = 0.09123
8h RMSE = 0.116066
16h MAE = 0.105653
16h RMSE = 0.140229
24h MAE = 0.115149
24h RMSE = 0.146106
All MAE = 0.103696
All RMSE = 0.134266
========= rcnn_24h =========
1h MAE = 0.091164
1h RMSE = 0.118983
8h MAE = 0.095313
8h RMSE = 0.12262
16h MAE = 0.094327
16h RMSE = 0.122789
24h MAE = 0.112184
24h RMSE = 0.143477
All MAE = 0.096698
All RMSE = 0.125637


In [43]:
models = ['mlp', 'rnn', 'lstm', 'cnn', 'rcnn_24h']
for i in range(5):
    saved_model = load_model('saved_model/{}.h5'.format(models[i]))
    yhat = saved_model.predict(test_X)
    inv_yhat = scaler.inverse_transform(yhat)
    inv_y = scaler.inverse_transform(test_y)
    inv_yhat = pd.DataFrame(inv_yhat)
    inv_y = pd.DataFrame(inv_y)
    error = inv_y - inv_yhat
    inv_yhat.to_csv('output/inv_yhat_' + models[i] + '_noise.csv')